<a href="https://colab.research.google.com/github/RGologorsky/fastmri/blob/master/01_kspace_tfms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kspace tfms

- Implemented kspace transforms with np and Pytorch tensors
- Testing in testing_01_kspace_tfms.ipynb.

For real images, 
- FFT of a purely real array (eg image) has conjugate symmetry: ```
fft2_result[a, b] = fft2_result[-a, -b].conj()
```
- So rfft2 ouputs the left half (plus one column) of fft2 to save space & memory

- RFFT output could correspond to either an odd or even length signal.
- By default, irfft assumes an even output length.
- To avoid losing information, when doing inverse (irfft), given the correct length of the original input.

Sources: 

- https://stackoverflow.com/questions/43001729/how-should-i-interpret-the-output-of-numpy-fft-rfft2

- https://numpy.org/doc/stable/reference/generated/numpy.fft.irfft.html

In [0]:
class TensorTfmsBase():

  # convert image to Pytorch tensor, real tensor into complex tensor
  @staticmethod
  def im2arr(im):      return tensor(im).double()
  
  @staticmethod
  def real2complex(t): return torch.stack((t, torch.zeros(t.shape).double()), axis=-1)

  # (centered) complex arr obj to (centered) kspace (and back)
  complex2k  = [T.fft2]
  k2complex  = [T.to_tensor, T.ifft2]
   
  # Viz complex k in magnitude-only kspace (log)
  log_magn = [T.complex_abs, add(1e-9), torch.log, torch.abs]

  # FFT & Shifting
  fftshift  = T.fftshift
  ifftshift = T.ifftshift

  # real to centered kspace (and back)
  @classmethod
  def real2k(cls): return [cls.fft, cls.fftshift]
  @classmethod
  def k2real(cls): return [cls.ifftshift, cls.ifft]


In [0]:
class TensorTfms(TensorTfmsBase):
 fft = T.fft2

 # ifft = ifft2 + magnitude
 def ifft(x): return T.complex_abs(T.ifft2(x))


In [0]:
class RealTensorTfms(TensorTfmsBase):
   # real to centered kspace (and back)
  @classmethod
  def real2k(cls, onesided=True):
     cls.fft = partial(torch.rfft, signal_ndim = 2, normalized=True, onesided=onesided)
     return super().real2k()

  @classmethod
  def k2real(cls, onesided=True, s=None): 
    cls.ifft =  partial(torch.irfft, signal_ndim = 2, normalized=True, onesided=onesided, signal_sizes=s)
    return super().k2real()


In [0]:
class BatchShift():
  # batch fft & ifft shift -- shift all but batch dimension
  def ifftshift(x):
    dim = tuple(range(x.dim()))[1:]
    shift = [(dim + 1) // 2 for dim in x.shape[1:]]
    return T.roll(x, shift, dim)

  def fftshift(x):
    dim = tuple(range(x.dim()))[1:]
    shift = [dim // 2 for dim in x.shape[1:]]
    return T.roll(x, shift, dim)

class BatchTensorTfms(TensorTfms, BatchShift): pass
class BatchRealTensorTfms(RealTensorTfms, BatchShift): pass

In [0]:
def apply(data, tfms, pre=None, post=None): return Pipeline(L(pre) + L(tfms) + L(post))(data)

## Viz: function to plot images

In [0]:
def idx(lst,i, default=None): return lst[i] if i < len(lst) else default

In [0]:
def plot(imgs, titles=[None], cmaps=["gray"], nrows=1, ncols=1, figsize = (6,6), **kwargs):

   # listify so we input can be string instead of 1-item list
  imgs, titles, cmaps = L(imgs), L(titles), L(cmaps)

  # default set nrows, ncols = 1, len(imgs)
  if nrows * ncols != len(imgs): nrows, ncols = 1, len(imgs)

  # default repeat cmap until same size as images
  cmaps = cmaps * int(len(imgs)/len(cmaps))

  fig,axes = plt.subplots(nrows, ncols, figsize=figsize, squeeze=False)
  axes = axes.flatten()
  for i,im in enumerate(imgs): 
    axes[i].imshow(im, cmap=idx(cmaps,i))
    axes[i].set_xticklabels([]), axes[i].set_yticklabels([])
    axes[i].set_title(idx(titles,i))
    axes[i].set_xlabel(im.shape)
  fig.tight_layout()
  fig.show()

# Common KSpace Transforms

In [0]:
# permute kspace tensor: N1HW(Complex) to N(Complex)HW
class Complex2Channel(Transform):
  order = 99 # happens after complex k

  # N1HW(Complex) -> N(Complex)HW
  def encodes(self, t:Tensor):
    if t.size(-1) == 2: return torch.squeeze(t.transpose(-1,-2).transpose(-2,-3))
    return t

  # NCHW -> NHWC
  def decodes(self, t:Tensor):
    if t.size(-3) == 2: return t.transpose(-3,-2).transpose(-2,-1)
    return t
    
# shows concat real & kspace into one image
class ShowK(Tuple):
  def show(self, ctx=None, **kwargs): 
    k,real = self
    line = k.new_zeros(k.shape[0], 10)
    return show_image(torch.cat([k,line,real], dim=1), title = "K & Real", ctx=ctx, **kwargs)

# take dataset item (real img, category), convert to (k arr, category)
class BatchReal2ComplexK(Transform):
  order = 50 # needs to run after save shape

  # do nothing to tensor categories
  def encodes(self, t:TensorCategory): return t
  def decodes(self, t:TensorCategory): return t

  def encodes(self, t:Tensor): return apply(t, TensorTfms.batch_real2k(onesided=False))


  def decodes(self, t_k:Tensor):
    t_k_abs         = apply(t_k, TensorTfms.t_abs)
    t_k_log_abs     = apply(t_k_abs, TensorTfms.log_abs)

    t_real     = apply(t_k, TensorTfms.batch_k2real(onesided=False))
    
    return ShowK(t_k_log_abs, t_real)

# converts complex k-space (2channel) to amplitude k-space (1channel)
class ComplexK2LogAbs(Transform):
  order = 51 # needs to run after Real2ComplexK

  # do nothing to tensor categories
  def encodes(self, t:TensorCategory): return t
  def decodes(self, t:TensorCategory): return t

  def encodes(self, t:Tensor): return apply(t, TensorTfms.log_abs, pre=TensorTfms.t_abs)